<a href="https://colab.research.google.com/github/TOM-BOHN/SFDC-User-Permissions-AI/blob/main/Notebooks/SFDC_User_Permission_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Install the Python SDK.

In [14]:
!git clone https://github.com/TOM-BOHN/SFDC-User-Permissions-AI.git

Cloning into 'SFDC-User-Permissions-AI'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 33 (delta 9), reused 23 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 12.93 KiB | 6.47 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [6]:
!pip install -Uq "google-genai==1.7.0"

In [13]:
from google import genai
from google.genai import types

from IPython.display import Markdown, display

genai.__version__

###################################

import pandas as pd
import enum

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [8]:
from google.colab import userdata

client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

### Automated retry

This codelab sends a lot of requests, so set up an automatic retry
that ensures your requests are retried when per-minute quota is reached.

In [9]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

In [12]:
url = "https://raw.githubusercontent.com/TOM-BOHN/SFDC-User-Permissions-AI/refs/heads/main/Inputs/User_Permission_Reference_Data__Sample.csv"
df = pd.read_csv(url)
df.head()

,Permission Name,API Name,Description
0,Access Data Cloud Data Explorer,AccessCdpDataExplorer,Allows user access Data Cloud Data Explorer.
1,Administer territory operations,ManageTerritories,Prerequisite user permission for a user to man...
2,Allow sending of List Emails,ListEmailSend,"Allow users to create, edit and send List Emails"
3,Api Only User,ApiUserOnly,Access Salesforce.com only through a Salesforc...
4,Author Apex,AuthorApex,Create Apex classes and triggers.


In [17]:
with open('/content/SFDC-User-Permissions-AI/Prompts/prompt_user_perm_risk_rating.md', 'r') as f:
    prompt_user_perm_risk_rating = f.read()

#print(prompt_user_perm_risk_rating)

In [28]:
total_records = len(df)

for i in range(2):
  print(f'Analyzing Permission {i+1} of {total_records}...')
  print('Name:       ', df['Permission Name'][i])
  print('API Name:   ', df['API Name'][i])
  print('Description:', df['Description'][i])
  print('--------------------')

  # Substitute variables into the prompt
  prompt_with_vars = prompt_user_perm_risk_rating.format(
    permission_name=df['Permission Name'][i],
    permission_description=df['Description'][i]
  )

Analyzing Permission 1 of 58...
Name:        Access Data Cloud Data Explorer
API Name:    AccessCdpDataExplorer
Description: Allows user access Data Cloud Data Explorer.
--------------------
Analyzing Permission 2 of 58...
Name:        Administer territory operations
API Name:    ManageTerritories
Description: Prerequisite user permission for a user to manage a territory branch.
--------------------
